<a href="https://colab.research.google.com/github/swastome-official/SW09-01-Development-of-an-AI-Based-Support-System-for-Brain-Tumor-Detection-Using-CT-and-MRI-Images/blob/main/SW09_01_Development_of_an_AI_Based_Support_System_for_Brain_Tumor_Detection_Using_CT_and_MRI_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Define paths to training and testing directories
dataset_path = "/path/to/brain-tumor-mri-dataset"  # Update with actual path after downloading
train_dir = os.path.join(dataset_path, "Training")
test_dir = os.path.join(dataset_path, "Testing")

# Image preprocessing function
def preprocess_image(image_path, target_size=(224, 224)):
    try:
        # Read image in grayscale
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            return None
        # Basic quality check: Remove images with extreme intensity (e.g., nearly all black/white)
        if np.std(img) < 10 or np.mean(img) < 20 or np.mean(img) > 235:
            return None
        # Resize image
        img = cv2.resize(img, target_size)
        # Normalize pixel values to [0, 1]
        img = img / 255.0
        return img
    except:
        return None

# Load and preprocess dataset
def load_dataset(directory, target_size=(224, 224)):
    images = []
    labels = []
    class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']

    for class_name in class_names:
        class_dir = os.path.join(directory, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = preprocess_image(img_path, target_size)
            if img is not None:
                images.append(img)
                # Encode labels: 0=glioma, 1=meningioma, 2=notumor, 3=pituitary
                labels.append(class_names.index(class_name))

    images = np.array(images)
    labels = np.array(labels)
    # Reshape images to include channel dimension
    images = images.reshape(-1, target_size[0], target_size[1], 1)
    return images, labels, class_names

# Load training and testing data
train_images, train_labels, class_names = load_dataset(train_dir)
test_images, test_labels, _ = load_dataset(test_dir)

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation/test data generator (no augmentation)
test_datagen = ImageDataGenerator()

# Convert labels to categorical
train_labels_cat = tf.keras.utils.to_categorical(train_labels, num_classes=4)
test_labels_cat = tf.keras.utils.to_categorical(test_labels, num_classes=4)

# Create CNN model
def create_cnn_model(input_shape=(224, 224, 1), num_classes=4):
    model = Sequential([
        # First convolutional block
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        # Second convolutional block
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        # Third convolutional block
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        # Flatten and dense layers
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Initialize and compile model
model = create_cnn_model()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model summary
model.summary()

# Train the model
batch_size = 32
epochs = 20

history = model.fit(
    train_datagen.flow(train_images, train_labels_cat, batch_size=batch_size),
    epochs=epochs,
    validation_data=test_datagen.flow(test_images, test_labels_cat, batch_size=batch_size),
    verbose=1
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_images, test_labels_cat, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Generate predictions
y_pred = model.predict(test_images)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = test_labels

# Classification report
print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Plot training history
plt.figure(figsize=(12, 4))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Save the model
model.save('brain_tumor_cnn_model.h5')